In [1]:
%pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from operator import add, abs


spark=SparkSession.builder.config("spark.driver.memory", "15g").appName("iic2440").getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=iic2440>

going to assume that nodes always start on 1 and don't skip numbers

In [3]:
edges = [(1, 2, 10), (2, 3, 3), (2, 4, 24), (3, 2, 1)]

In [4]:
def single_source_shortest_path(edges, initialNode = 1):
  rdd_edges = sc.parallelize(edges)
  rdd_accumulated = rdd_edges.flatMap(lambda x: x[:2]).distinct().map(lambda n: (n, 0) if n == initialNode else (n, float('inf')))

  delta = 1
  i = 0
  while delta > 0:
    rdd_final_msgs = rdd_edges.map(lambda x: (x[0], (x[1], x[2]))).join(rdd_accumulated).values().map(lambda x: (x[0][0], x[0][1] + x[1])).reduceByKey(lambda x,y: x if x < y else y)
    new_rdd_accumulated = rdd_accumulated.leftOuterJoin(rdd_final_msgs).mapValues(lambda l: [float('inf') if x is None else x for x in l]).mapValues(lambda x: x[0] if x[0] < x[1] else x[1])
    delta = rdd_accumulated.join(new_rdd_accumulated).mapValues(lambda x: x[1] != x[0]).values().sum() #this is 0 if there were no changes
    rdd_accumulated = new_rdd_accumulated
    i += 1
    print('iter', i)
    print('delta', delta)
    print('-----')
  return rdd_accumulated.collect()

def single_source_shortest_path_non_optimized(edges, initialNode = 1):
  rdd_edges = sc.parallelize(edges)
  rdd_nodes = rdd_edges.flatMap(lambda x: x[:2]).distinct()
  rdd_accumulated = rdd_nodes.map(lambda n: (n, 0) if n == initialNode else (n, float('inf')))

  delta = 1
  i = 0
  while delta > 0:
    rdd_edges_prejoin = rdd_edges.map(lambda x: (x[0], (x[1], x[2])))
    rdd_all_msgs = rdd_edges_prejoin.join(rdd_accumulated).values().map(lambda x: (x[0][0], x[0][1] + x[1]))
    rdd_final_msgs = rdd_all_msgs.reduceByKey(lambda x,y: x if x < y else y)
    new_rdd_accumulated = rdd_accumulated.leftOuterJoin(rdd_final_msgs).mapValues(lambda l: [float('inf') if x is None else x for x in l]).mapValues(lambda x: x[0] if x[0] < x[1] else x[1])
    delta = rdd_accumulated.join(new_rdd_accumulated).mapValues(lambda x: x[1] != x[0]).values().sum() #this is 0 if there were no changes
    rdd_accumulated = new_rdd_accumulated
    i += 1
    print('iter', i)
    print('delta', delta)
    print('-----')
  return rdd_accumulated.collect()


In [5]:
single_source_shortest_path(edges)

iter 1
delta 1
-----
iter 2
delta 2
-----
iter 3
delta 0
-----


[(1, 0), (2, 10), (3, 13), (4, 34)]